# Julia言語で入門するプログラミング

[Julia言語で入門するプログラミング](https://muuumin.net/julia-intro-prog/)

## 条件分岐

### HP計算の修正
前回のコードで、HP計算は単純に `現在のHP - ダメージ量` を計算していた

しかしこの計算では、例えば現在のHPが 10 で、ダメージ量が 20 の場合、HPが -10 になってしまう

こういった場合、HPは 0 になるのが普通のゲームである

そのため、`ダメージ後のHPが 0 未満になる場合、ダメージ後のHPを 0 とする` という条件分岐処理を追加する

In [1]:
using Test

"""
    HP計算(HP::Int, ダメージ量::Int) -> ダメージ後のHP::Int

HP計算関数
"""
function HP計算(HP::Int, ダメージ量::Int)
    ダメージ後のHP = HP - ダメージ量
    if ダメージ後のHP < 0
        return 0
    end
    return ダメージ後のHP
end

# テスト実行
@testset "HP計算" begin
    @test HP計算(20, 10) === 10 # HP: 20, ダメージ量: 10 が入力されれば 10 が出力されるべき
    @test HP計算(10, 10) === 0  # HP: 10, ダメージ量: 10 が入力されれば 0 が出力されるべき
    @test HP計算(0, 10) === 0   # HP: 0, ダメージ量: 10 が入力されれば 0 が出力されるべき (-10 < 0 のため)
end

Test Summary: | Pass  Total
HP計算        |    3      3


Test.DefaultTestSet("HP計算", Any[], 3, false, false)

### ダメージ計算の修正
ダメージ計算の関数は、現状、攻撃力、防御力を引数にとりダメージ量を返している

一旦実装の詳細は忘れて、ダメージ計算関数に期待する振る舞いを考える

- 正常系 (攻撃力を防御力で割って10倍して四捨五入した値をダメージ量とする)
    - 攻撃力 10, 防御力 10 のときに、ダメージ量は 10 になって欲しい
    - 攻撃力 15, 防御力 100 のときに、ダメージ量は 2 になって欲しい
    - 攻撃力 14, 防御力 100 のときに、ダメージ量は 1 になって欲しい
    - 攻撃力 0, 防御力 10 のときに、ダメージ量は 0 になって欲しい
- 異常系 (例外: 不正な動きと判断して処理を中断する)
    - 攻撃力 -1, 防御力 10 のときに、例外を発生させて欲しい
    - 攻撃力 10, 防御力 -1 のときに、例外を発生させて欲しい
    - 攻撃力 -1, 防御力 -1 のときに、例外を発生させて欲しい
    - 攻撃力 10, 防御力 0 のときに、例外を発生させて欲しい

これを自動テストとして実装する

```julia
@testset "ダメージ計算" begin
    # 正常系テスト
    @test ダメージ計算(10, 10) == 10
    @test ダメージ計算(15, 100) == 2
    @test ダメージ計算(14, 100) == 1
    @test ダメージ計算(0, 100) == 0

    # 異常系テスト
    ## @test_throws 例外の種類 関数呼び出し: 例外発生をテストする
    @test_throws DomainError ダメージ計算(-1, 10)
    @test_throws DomainError ダメージ計算(10, -1)
    @test_throws DomainError ダメージ計算(-1, -1)
    @test_throws DomainError ダメージ計算(10, 0)
end
```

このようなテストを通るようにダメージ計算関数を実装する

In [2]:
"""
    ダメージ計算(攻撃力::Int, 防御力::Int) -> ダメージ量::Int

ダメージ計算関数
"""
function ダメージ計算(攻撃力::Int, 防御力::Int)
    if 攻撃力 < 0 || 防御力 < 0
        throw(DomainError((攻撃力, 防御力), "攻撃力, 防御力は正の値である必要があります")) 
    end
    if 防御力 === 0
        throw(DomainError(防御力, "防御力は0より大きい値である必要があります")) 
    end
    ダメージ量 = round(Int, 10 * 攻撃力 / 防御力)
end

@testset "ダメージ計算" begin
    # 正常系テスト
    @test ダメージ計算(10, 10) == 10
    @test ダメージ計算(15, 100) == 2
    @test ダメージ計算(14, 100) == 1
    @test ダメージ計算(0, 100) == 0

    # 異常系テスト
    ## @test_throws 例外の種類 関数呼び出し: 例外発生をテストする
    @test_throws DomainError ダメージ計算(-1, 10)
    @test_throws DomainError ダメージ計算(10, -1)
    @test_throws DomainError ダメージ計算(-1, -1)
    @test_throws DomainError ダメージ計算(10, 0)
end

Test Summary: | Pass  Total
ダメージ計算  |    8      8


Test.DefaultTestSet("ダメージ計算", Any[], 8, false, false)

## モンスターに攻撃チャンスを与える

前回のプログラムでは、勇者がモンスターを一方的に攻撃するだけだった

今回は、モンスターにも攻撃するチャンスを与える

In [3]:
function main()
    モンスターHP = 30
    モンスター攻撃力 = 10 # 追加
    モンスター防御力 = 10
    勇者HP = 30          # 追加
    勇者攻撃力 = 10
    勇者防御力 = 10       # 追加

    println("モンスターに遭遇した！")
    println("戦闘開始！")

    for _ in 1:3
        println("----------")
        println("勇者の攻撃！")
        モンスターダメージ = ダメージ計算(勇者攻撃力, モンスター防御力)
        モンスターHP = HP計算(モンスターHP, モンスターダメージ)
        println("モンスターは $(モンスターダメージ) のダメージを受けた！")
        println("モンスターの残りHP：$(モンスターHP)")

        # 追加: モンスターの攻撃
        println("----------")
        println("モンスターの攻撃！")
        勇者ダメージ = ダメージ計算(モンスター攻撃力, 勇者防御力)
        勇者HP = HP計算(勇者HP, 勇者ダメージ)
        println("勇者は $(勇者ダメージ) のダメージを受けた！")
        println("勇者の残りHP：$(勇者HP)")
    end

    println("戦闘に勝利した！")
end

main()

モンスターに遭遇した！
戦闘開始！
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：20
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：20
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：10
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：10
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：0
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：0
戦闘に勝利した！


## プログラムの改良

とりあえず上記までで、モンスターが攻撃できるようになった

しかしこのプログラムでは、勇者が先にモンスターのHPを 0 にするが、モンスターは倒れずに勇者に攻撃を行い、両者ともにHP 0 で終了する

また、必ず勇者が先制攻撃してしまうため、絶対に勇者が勝つ面白くないゲームになってしまっている

そこで、以下のような改良を加えることにする

1. HPが 0 になっても攻撃できるのはおかしいため、いずれかのHPが 0 になった時点で戦闘を終了する
2. 常に勇者が先制攻撃するのは不公平なため、勇者とモンスターの攻撃順はランダムになるようにする

### whileによる条件ループ
ここまでの戦闘は for 文により常に3回行動して終了するプログラムになっていた

しかし、`いずれかのHPが 0 になった時点で戦闘を終了する` のが自然な戦闘の流れであるため、これを while 文に書き換え `勇者のHPが 0 より大きい かつ モンスターのHPが 0 より大きい場合にループを継続する` という形に書き換える

また、それぞれの攻撃が終わった時点で攻撃を受けた側のHPを確認し、`HPが 0 であればループを抜ける` という条件を加える

これらの変更により `いずれかのHPが 0 になった時点で戦闘を終了する` 処理を実現できる

In [4]:
function main()
    モンスターHP = 30
    モンスター攻撃力 = 10
    モンスター防御力 = 10
    勇者HP = 30
    勇者攻撃力 = 10
    勇者防御力 = 10

    println("モンスターに遭遇した！")
    println("戦闘開始！")

    # 戦闘ループの条件: 勇者のHPが 0 より大きい かつ モンスターのHPが 0 より大きい
    while 勇者HP > 0 && モンスターHP > 0
        println("----------")
        println("勇者の攻撃！")
        モンスターダメージ = ダメージ計算(勇者攻撃力, モンスター防御力)
        モンスターHP = HP計算(モンスターHP, モンスターダメージ)
        println("モンスターは $(モンスターダメージ) のダメージを受けた！")
        println("モンスターの残りHP：$(モンスターHP)")

        # モンスターのHPを確認して、0 になっていたらループを抜ける (break)
        if モンスターHP === 0
            break
        end

        println("----------")
        println("モンスターの攻撃！")
        勇者ダメージ = ダメージ計算(モンスター攻撃力, 勇者防御力)
        勇者HP = HP計算(勇者HP, 勇者ダメージ)
        println("勇者は $(勇者ダメージ) のダメージを受けた！")
        println("勇者の残りHP：$(勇者HP)")

        # 勇者のHPを確認して、0 になっていたらループを抜ける (break)
        if 勇者HP === 0
            break
        end
    end

    println("戦闘に勝利した！")
end

main()

モンスターに遭遇した！
戦闘開始！
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：20
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：20
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：10
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：10
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：0
戦闘に勝利した！


### 乱数の導入
乱数とはランダムに生成される数のことである

乱数を使うことで、勇者とモンスターの攻撃順をランダムになるようにして、勇者が常に先制攻撃するのを改良する

Julia では乱数生成関数として `rand()` が用意されている

`rand()` 関数は 0 ~ 1 の間の小数をランダムに生成する

In [5]:
rand() |> println
rand() |> println
rand() |> println

0.9247761880702967
0.37936004470935825
0.9032638460401372


このとき `rand() < 0.5` となる確率は50％、`rand() < 0.3` となる確率は30％である

今回、勇者とモンスターは対等に扱いたいため、50％の確率で勇者が、50％の確率でモンスターが先制攻撃することにする

In [6]:
function main()
    モンスターHP = 30
    モンスター攻撃力 = 10
    モンスター防御力 = 10
    勇者HP = 30
    勇者攻撃力 = 10
    勇者防御力 = 10

    println("モンスターに遭遇した！")
    println("戦闘開始！")

    while 勇者HP > 0 && モンスターHP > 0
        if rand() < 0.5
            # 50％の確率で勇者が先制攻撃
            println("----------")
            println("勇者の攻撃！")
            モンスターダメージ = ダメージ計算(勇者攻撃力, モンスター防御力)
            モンスターHP = HP計算(モンスターHP, モンスターダメージ)
            println("モンスターは $(モンスターダメージ) のダメージを受けた！")
            println("モンスターの残りHP：$(モンスターHP)")

            if モンスターHP === 0
                break
            end

            println("----------")
            println("モンスターの攻撃！")
            勇者ダメージ = ダメージ計算(モンスター攻撃力, 勇者防御力)
            勇者HP = HP計算(勇者HP, 勇者ダメージ)
            println("勇者は $(勇者ダメージ) のダメージを受けた！")
            println("勇者の残りHP：$(勇者HP)")

            if 勇者HP === 0
                break
            end
        else
            # 50％の確率でモンスターが先制攻撃
            println("----------")
            println("モンスターの攻撃！")
            勇者ダメージ = ダメージ計算(モンスター攻撃力, 勇者防御力)
            勇者HP = HP計算(勇者HP, 勇者ダメージ)
            println("勇者は $(勇者ダメージ) のダメージを受けた！")
            println("勇者の残りHP：$(勇者HP)")

            if 勇者HP === 0
                break
            end

            println("----------")
            println("勇者の攻撃！")
            モンスターダメージ = ダメージ計算(勇者攻撃力, モンスター防御力)
            モンスターHP = HP計算(モンスターHP, モンスターダメージ)
            println("モンスターは $(モンスターダメージ) のダメージを受けた！")
            println("モンスターの残りHP：$(モンスターHP)")

            if モンスターHP === 0
                break
            end
        end
    end

    # モンスターHPが 0 になっているかどうかで戦闘終了メッセージを変える
    if モンスターHP === 0
        println("戦闘に勝利した！")
    else
        println("戦闘に敗北した・・・")
    end
end

main()

モンスターに遭遇した！
戦闘開始！
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：20
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：20
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：10
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：10
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：0
戦闘に敗北した・・・


## リファクタリング

上記のコードは、攻撃プログラムをコピペして作られており、同じコードがプログラム中に何度も出てきている

こういったコードは、一部のコードを修正したい場合に、コピペした全てのコードを修正しなければならなくなり、非常に扱いづらい

また、コード自体も長くなりがちで可読性も悪い

そのため、ここからはコードを読みやすく保守しやすい形に改良していく

このように、プログラムの動作を変えずに内部の構造を改良することを **リファクタリング** と呼ぶ

### 自動テストしやすい関数を作る
リファクタリングで大事なことは動作を変えないことである

動作を変えないことを保証するために自動テストを作る

しかし、今回追加した乱数生成関数は、実行する度に出力値が変わる副作用関数である

副作用関数を関数の中で使うと、その関数も副作用汚染されてしまうため、副作用関数と純粋関数（参照透過性のある関数）は極力分離して定義するのが肝要である

今回、`main` 関数を自動テストしたいが、`main` 関数の中では `rand()` 副作用関数が使われている

この副作用汚染を防ぐのによく使われる手法としては、関数自体を関数の引数に渡す **高階関数** というものがある

```julia
# 引数に関数を渡せるようにする
function main(乱数生成関数::Function)
    ...
end

# ゲームを実行するときは rand() 関数を渡す
## 副作用汚染されるが、ゲームとしては自然な挙動になる
main(rand)

# テストを実行するときは副作用のない純粋な関数を渡す
## 純粋関数として実行できるので自動テストが可能
main(
    # 実行する度に 0.1 => 0.5 => 0.2 => 0.7 の順に数値を返す関数を生成して渡す
    ## - 疑似乱数生成関数: main 関数内部で rand() の代わりに呼び出される関数
    ##   * この関数も呼び出す度に出力値が変わるため厳密には副作用関数だが、出力値が事前にわかるため自動テストに使うことができる
    ## - 疑似乱数生成高階関数: 疑似乱数生成関数 を作成して返す高階関数
    (
        function 疑似乱数生成高階関数(疑似乱数配列::Vector{<:Number})
            function 疑似乱数生成関数()
                popfirst!(疑似乱数配列)
            end
        end
    )([0.1, 0.5, 0.2, 0.7])
)
```

また、テストを行いたい関数は、テストしたい値を出力値として返さなければならない

今回 `main` 関数でテストしたい値としては、「戦闘終了までにかかったターン数」と「戦闘終了時点での勇者とモンスターのHP」を採用することにする

In [7]:
"""
    main(乱数生成関数::Function) ->  戦闘ターン数::Int,
                                    戦闘後の勇者HP::Int,
                                    戦闘後のモンスターHP::Int

自動テスト可能な main 関数
"""
function main(乱数生成関数::Function)
    モンスターHP = 30
    モンスター攻撃力 = 10
    モンスター防御力 = 10
    勇者HP = 30
    勇者攻撃力 = 10
    勇者防御力 = 10
    戦闘ターン数 = 1

    println("モンスターに遭遇した！")
    println("戦闘開始！")

    while 勇者HP > 0 && モンスターHP > 0
        if 乱数生成関数() < 0.5
            # 50％の確率で勇者が先制攻撃
            println("----------")
            println("勇者の攻撃！")
            モンスターダメージ = ダメージ計算(勇者攻撃力, モンスター防御力)
            モンスターHP = HP計算(モンスターHP, モンスターダメージ)
            println("モンスターは $(モンスターダメージ) のダメージを受けた！")
            println("モンスターの残りHP：$(モンスターHP)")

            if モンスターHP === 0
                break
            end

            println("----------")
            println("モンスターの攻撃！")
            勇者ダメージ = ダメージ計算(モンスター攻撃力, 勇者防御力)
            勇者HP = HP計算(勇者HP, 勇者ダメージ)
            println("勇者は $(勇者ダメージ) のダメージを受けた！")
            println("勇者の残りHP：$(勇者HP)")

            if 勇者HP === 0
                break
            end
        else
            # 50％の確率でモンスターが先制攻撃
            println("----------")
            println("モンスターの攻撃！")
            勇者ダメージ = ダメージ計算(モンスター攻撃力, 勇者防御力)
            勇者HP = HP計算(勇者HP, 勇者ダメージ)
            println("勇者は $(勇者ダメージ) のダメージを受けた！")
            println("勇者の残りHP：$(勇者HP)")

            if 勇者HP === 0
                break
            end

            println("----------")
            println("勇者の攻撃！")
            モンスターダメージ = ダメージ計算(勇者攻撃力, モンスター防御力)
            モンスターHP = HP計算(モンスターHP, モンスターダメージ)
            println("モンスターは $(モンスターダメージ) のダメージを受けた！")
            println("モンスターの残りHP：$(モンスターHP)")

            if モンスターHP === 0
                break
            end
        end

        # 戦闘ターン数 を 1 増やす
        戦闘ターン数 += 1
    end

    # モンスターHPが 0 になっているかどうかで戦闘終了メッセージを変える
    if モンスターHP === 0
        println("戦闘に勝利した！")
    else
        println("戦闘に敗北した・・・")
    end

    # テストしたい値を返す
    戦闘ターン数, 勇者HP, モンスターHP
end

#=
自動テスト: 乱数生成関数が出力する数列をもとに、戦闘ターン数と戦闘後の勇者・モンスターのHPをテストする

- [0.5, 0.4, 0.6]:
    1. モンスター(HP: 30) -> 勇者(HP: 30) 攻撃 ==> 勇者(HP: 20) -> モンスター(HP: 30) 攻撃
    2. 勇者(HP: 20) -> モンスター(HP: 20) 攻撃 ==> モンスター(HP: 10) -> 勇者(HP: 20) 攻撃
    3. モンスター(HP: 10) -> 勇者(HP: 10) 攻撃 ==> 戦闘終了: 勇者(HP: 0), モンスター(HP: 10)
- [0.4, 0.7, 0.3]:
    1. 勇者(HP: 30) -> モンスター(HP: 30) 攻撃 ==> モンスター(HP: 20) -> 勇者(HP: 30) 攻撃
    2. モンスター(HP: 20) -> 勇者(HP: 20) 攻撃 ==> 勇者(HP: 10) -> モンスター(HP: 20) 攻撃
    3. 勇者(HP: 10) -> モンスター(HP: 10) ==> 戦闘終了: 勇者(HP: 10), モンスター(HP: 0)
=#
function 疑似乱数生成高階関数(疑似乱数配列::Vector{<:Number})
    function 疑似乱数生成関数()
        popfirst!(疑似乱数配列)
    end
end

@testset "main関数" begin
    @test main(疑似乱数生成高階関数([0.5, 0.4, 0.6])) === (3, 0, 10)
    @test main(疑似乱数生成高階関数([0.4, 0.7, 0.3])) === (3, 10, 0)
end

モンスターに遭遇した！
戦闘開始！
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：20
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：20
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：10
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：10
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：0
戦闘に敗北した・・・
モンスターに遭遇した！
戦闘開始！
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：20
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：20
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：10
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：10
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：0
戦闘に勝利した！
Test Summary: | Pass  Total
main関数      |    2      2


Test.DefaultTestSet("main関数", Any[], 2, false, false)

だいぶ不格好だが、ひとまず `main` 関数を自動テストできるようになった

次回からリファクタリングを行っていく